# 단일음정 유지 능력 평가를 위한 SPICE 기반 코드 (수정된 버전)

In [111]:
# 필요한 라이브러리 import
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import librosa
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

# TensorFlow 설정 최적화 (FailedPreconditionError 해결)
print("TensorFlow 버전:", tf.__version__)

# 1. GPU 메모리 증가 허용 (메모리 문제 해결)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU 메모리 증가 허용 설정 완료")
    except RuntimeError as e:
        print(f"GPU 설정 오류: {e}")
else:
    print("GPU 없음, CPU 사용")

# 2. TensorFlow 실행 모드 설정
tf.config.run_functions_eagerly(False)  # Graph 모드 사용
print("TensorFlow Graph 모드 설정 완료")

# 3. TensorFlow Hub 캐시 클리어 (캐시 문제 해결)
import tempfile
import shutil

def clear_tfhub_cache():
    """TensorFlow Hub 캐시를 클리어합니다"""
    try:
        cache_dir = os.path.join(tempfile.gettempdir(), 'tfhub_modules')
        if os.path.exists(cache_dir):
            shutil.rmtree(cache_dir)
            print("TensorFlow Hub 캐시 클리어 완료")
        else:
            print("TensorFlow Hub 캐시 없음")
    except Exception as e:
        print(f"캐시 클리어 오류: {e}")

# 캐시 클리어 실행
clear_tfhub_cache()

print("TensorFlow 환경 설정 완료")

TensorFlow 버전: 2.13.0
GPU 설정 오류: Physical devices cannot be modified after being initialized
TensorFlow Graph 모드 설정 완료
TensorFlow Hub 캐시 없음
TensorFlow 환경 설정 완료


In [112]:
# 1. 오디오 로드 함수
def load_audio(filepath, sample_rate=16000):
    """오디오 파일을 로드하고 정규화"""
    audio,sr = librosa.load(filepath, sr=sample_rate)
    # 정규화: [-1, 1] 범위로 조정
    if np.max(np.abs(audio)) > 0:
        audio = audio / np.max(np.abs(audio))
    return audio,sr

filepath = "0_아/e_1_1.wav"
print(f"현재 파일:{filepath}")


현재 파일:0_아/e_1_1.wav


In [113]:
# 2. SPICE 모델 전용 피치 예측 (FailedPreconditionError 해결)
def estimate_pitch_spice_only(audio,sr=16000):
    """SPICE 모델을 사용한 피치 추정 (초기화 문제 해결)"""
    try:
        print("SPICE 모델 로딩 중...")
        
        # TensorFlow 세션 완전 초기화
        tf.keras.backend.clear_session()  # 이전 세션 정리
        
        # TensorFlow Hub 모델 로드 (새로운 세션에서)
        with tf.device('/CPU:0'):  # CPU 강제 사용으로 GPU 문제 회피
            model = hub.load("https://tfhub.dev/google/spice/2")
        
        print("SPICE 모델 로드 완료")
        
        # 오디오 정규화 확인
        if np.max(np.abs(audio)) == 0:
            raise ValueError("오디오에 신호가 없습니다")
        
        # TensorFlow 텐서로 변환 (정확한 dtype과 shape)
        audio_tensor = tf.convert_to_tensor(audio, dtype=tf.float32)
        print(f"오디오 텐서 생성: shape={audio_tensor.shape}, dtype={audio_tensor.dtype}")
        
        # 모델 변수 초기화 강제 실행
        print("SPICE 모델 변수 초기화 중...")
        
        # 명시적으로 CPU에서 실행 (GPU 초기화 문제 회피)
        with tf.device('/CPU:0'):
            # 모델 서명 확인
            signature_keys = list(model.signatures.keys())
            # print(f"사용 가능한 서명: {signature_keys}")
            
            # 더미 입력으로 모델 워밍업 (변수 초기화 유도)
            print("변수 초기화 중...")
            dummy_audio = tf.zeros([1000], dtype=tf.float32)  # 더미 오디오
            try:
                _ = model.signatures["serving_default"](dummy_audio)
                print("초기화 성공")
            except Exception as warmup_error:
                print(f"초기화 실패: {warmup_error}")
            
            # 실제 모델 실행
            print("SPICE 모델 실행 중...")
            outputs = model.signatures["serving_default"](audio_tensor)
            
            # 결과 추출
            pitch = outputs["pitch"].numpy().flatten()
            uncertainty = outputs["uncertainty"].numpy().flatten()
            confidence = 1.0 - uncertainty
        
        print(f"SPICE 모델 실행 완료:")
        print(f"  - 피치 배열 크기: {len(pitch)}")
        print(f"  - 평균 신뢰도: {np.mean(confidence):.3f}")
        print(f"  - 유효한 피치 개수: {np.sum(pitch > 0)}")
        
        return pitch, confidence
        
    except Exception as e:
        print(f"❌ SPICE 모델 실행 실패: {e}")
        print("SPICE 모델 실행에 실패했습니다.")
        print("\nFailedPreconditionError 해결 방법:")
        print("1. TensorFlow 버전 확인: pip install tensorflow==2.12.0")
        print("2. TensorFlow Hub 재설치: pip install --upgrade tensorflow-hub")
        print("3. 시스템 재부팅 후 재시도")
        print("4. 다른 환경에서 테스트")
        print("\n프로그램을 종료합니다. 환경설정을 확인해 주세요")
        raise e

print("SPICE 전용 피치 추정 함수들 정의 완료 (FailedPreconditionError 해결 버전 + 세션 초기화)")

audio = load_audio(filepath)
print(f"현재 파일: {filepath}")

SPICE 전용 피치 추정 함수들 정의 완료 (FailedPreconditionError 해결 버전 + 세션 초기화)
현재 파일: 0_아/e_1_1.wav


In [114]:
# 3. 신뢰도로 피치 필터링(초기값 0.7)
def filter_pitch(pitch, confidence, threshold=0.7):
    """신뢰도가 높은 피치만 필터링"""
    filtered = [p if c >= threshold else 0 for p, c in zip(pitch, confidence)]
    valid_count = sum(1 for p in filtered if p > 0)
    print(f"필터링 완료: 전체 {len(filtered)}개 중 유효한 피치 {valid_count}개")
    return filtered


In [115]:
# 4. 이동 표준편차 계산
def moving_std(seq, win=5):
    """이동 윈도우 표준편차 계산"""
    if len(seq) == 0:
        return []
    
    padded = np.pad(seq, (win//2,), mode='edge')
    std_values = []
    
    for i in range(len(seq)):
        window = padded[i:i+win]
        # 0이 아닌 값들만 사용해서 표준편차 계산
        valid_values = window[window > 0]
        if len(valid_values) > 1:
            std_values.append(np.std(valid_values))
        else:
            std_values.append(0.0)
    
    return std_values

print("이동 표준편차 함수 정의 완료")

이동 표준편차 함수 정의 완료


In [120]:
# 5. 단일음정 구간 판별 및 평가
def evaluate_pitch_stability(filtered_pitch, std_threshold=1.5, actual_duration=None):
    """피치 안정성 평가 (실제 오디오 길이 기반)"""
    if len(filtered_pitch) == 0:
        return 0, 0, 0, []
    
    # 이동 표준편차 계산
    pitch_std = moving_std(filtered_pitch, win=5)
    
    # 단일음정 구간 판별 (표준편차가 낮고 피치가 0이 아닌 구간)
    mono_flags = [s < std_threshold and p > 0 for s, p in zip(pitch_std, filtered_pitch)]
    
    # 시간 계산 (실제 오디오 길이 기반)
    if actual_duration is not None:
        # 실제 프레임률 계산
        actual_fps = len(filtered_pitch) / actual_duration
        mono_duration = sum(mono_flags) / actual_fps
        total_duration = actual_duration  # 실제 오디오 길이 사용
        print(f"실제 프레임률: {actual_fps:.1f}fps (SPICE 내부 처리)")
    else:
        # 기본값 (100fps 가정)
        mono_duration = sum(mono_flags) / 100
        total_duration = len(filtered_pitch) / 100
    
    stable_ratio = mono_duration / total_duration if total_duration > 0 else 0
    
    print(f"안정성 평가 완료:")
    print(f"  - 단일음정 프레임: {sum(mono_flags)}개")
    print(f"  - 전체 프레임: {len(filtered_pitch)}개")
    print(f"  - 실제 총 길이: {total_duration:.2f}초")
    print(f"  - 단일음정 시간: {mono_duration:.2f}초")
    
    return stable_ratio, mono_duration, total_duration, mono_flags

print("안정성 평가 함수 정의 완료")


안정성 평가 함수 정의 완료


In [117]:
# 6. 전체 파이프라인 함수 (SPICE 전용)
def analyze_pitch_stability(filepath):
    """SPICE 전용 피치 안정성 분석 파이프라인"""
    print(f"\n=== SPICE 전용 피치 안정성 분석 시작: {filepath} ===")
    
    try:
        # 1. 오디오 로드
        audio, sr = load_audio(filepath)
        actual_duration = len(audio) / sr  # 실제 오디오 길이
        print(f"오디오 로드 완료: 길이 {actual_duration:.2f}초, 샘플레이트 {sr}Hz")
        
        # 2. SPICE로 피치 추정 (실패시 종료)
        pitch, confidence = estimate_pitch_spice_only(audio, sr)
        
        # 3. 피치 필터링
        filtered_pitch = filter_pitch(pitch, confidence)
        
        # 4. 안정성 평가 (실제 오디오 길이 전달)
        stable_ratio, mono_duration, total_duration, mono_flags = evaluate_pitch_stability(
            filtered_pitch, actual_duration=actual_duration
        )
        
        # 5. 결과 출력
        print(f"\n=== SPICE 분석 결과 ===")
        print(f"실제 오디오 길이: {actual_duration:.2f}초")
        print(f"SPICE 출력 프레임: {len(pitch)}개")
        print(f"단일음정 유지 시간: {mono_duration:.2f}초")
        print(f"안정성 비율: {stable_ratio:.2%}")
        
        # 6. 시각화 (실제 시간축 사용)
        try:
            plt.figure(figsize=(15, 8))
            
            # 실제 프레임률 계산
            actual_fps = len(pitch) / actual_duration
            
            # 서브플롯 1: 원본 피치
            plt.subplot(3, 1, 1)
            time_axis = np.arange(len(pitch)) / actual_fps  # 실제 시간축
            plt.plot(time_axis, pitch, 'b-', alpha=0.7, label='SPICE Raw Pitch')
            plt.ylabel('Pitch (Hz)')
            plt.title(f'SPICE Raw Pitch Estimation (실제 {actual_fps:.1f}fps)')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            # 서브플롯 2: 필터링된 피치
            plt.subplot(3, 1, 2)
            time_axis_filtered = np.arange(len(filtered_pitch)) / actual_fps
            plt.plot(time_axis_filtered, filtered_pitch, 'g-', linewidth=2, label='SPICE Filtered Pitch')
            plt.ylabel('Pitch (Hz)')
            plt.title('SPICE Filtered Pitch (High Confidence Only)')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            # 서브플롯 3: 안정성 분석
            plt.subplot(3, 1, 3)
            if len(filtered_pitch) > 0:
                pitch_std = moving_std(filtered_pitch)
                plt.plot(time_axis_filtered, pitch_std, 'r-', label='Moving Std', alpha=0.7)
                plt.axhline(y=1.5, color='orange', linestyle='--', label='Stability Threshold')
                
                # 안정한 구간 표시
                stable_regions = np.array(mono_flags) * max(pitch_std) * 0.1
                plt.fill_between(time_axis_filtered, 0, stable_regions, 
                               alpha=0.3, color='green', label='Stable Regions')
            
            plt.xlabel('Time (seconds)')
            plt.ylabel('Pitch Std')
            plt.title('SPICE Pitch Stability Analysis')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            print(f"시각화 오류: {e}")
        
        return stable_ratio
        
    except Exception as e:
        print(f"❌ SPICE 분석 실패: {e}")
        print("SPICE 모델 실행에 실패했습니다.")
        print("사용자 요청에 따라 librosa 대체 없이 프로그램을 종료합니다.")
        raise e

print("SPICE 전용 분석 파이프라인 함수 정의 완료 (실제 시간축 사용)")

SPICE 전용 분석 파이프라인 함수 정의 완료 (실제 시간축 사용)


In [ ]:
# 하이퍼파라미터 테스트 함수
def test_hyperparameters(filepath, std_threshold=1.5, confidence_threshold=0.7, window_size=5):
    """하이퍼파라미터를 조정해서 테스트"""
    print(f"\n=== 하이퍼파라미터 테스트 ===")
    print(f"std_threshold: {std_threshold}")
    print(f"confidence_threshold: {confidence_threshold}")  
    print(f"window_size: {window_size}")
    print("=" * 40)
    
    try:
        # 1. 오디오 로드
        audio, sr = load_audio(filepath)
        actual_duration = len(audio) / sr
        print(f"오디오 로드: {actual_duration:.2f}초")
        
        # 2. SPICE 피치 추정
        pitch, confidence = estimate_pitch_spice_only(audio, sr)
        
        # 3. 조정된 파라미터로 필터링
        filtered = [p if c >= confidence_threshold else 0 for p, c in zip(pitch, confidence)]
        valid_count = sum(1 for p in filtered if p > 0)
        print(f"필터링: {len(filtered)}개 중 {valid_count}개 유효")
        
        # 4. 조정된 윈도우로 표준편차 계산
        def custom_moving_std(seq, win):
            if len(seq) == 0:
                return []
            padded = np.pad(seq, (win//2,), mode='edge')
            std_values = []
            for i in range(len(seq)):
                window = padded[i:i+win]
                valid_values = window[window > 0]
                if len(valid_values) > 1:
                    std_values.append(np.std(valid_values))
                else:
                    std_values.append(0.0)
            return std_values
        
        pitch_std = custom_moving_std(filtered, window_size)
        
        # 5. 조정된 임계값으로 안정성 판단
        mono_flags = [s < std_threshold and p > 0 for s, p in zip(pitch_std, filtered)]
        
        # 6. 결과 계산
        actual_fps = len(filtered) / actual_duration
        mono_duration = sum(mono_flags) / actual_fps
        stable_ratio = mono_duration / actual_duration
        
        print(f"\n=== 결과 ===")
        print(f"단일음정 프레임: {sum(mono_flags)}개/{len(filtered)}개")
        print(f"단일음정 시간: {mono_duration:.2f}초")
        print(f"안정성 비율: {stable_ratio:.2%}")
        print(f"목표(10초)와의 차이: {abs(mono_duration - 10.0):.2f}초")
        
        return mono_duration, stable_ratio
        
    except Exception as e:
        print(f"테스트 실패: {e}")
        return 0, 0

# 여러 파라미터 조합 테스트

print("파라미터 조합 테스트 시작...")

# 조합 1: 기본값 (현재)
test_hyperparameters(filepath, std_threshold=1.5, confidence_threshold=0.7, window_size=5)

# 조합 2: std_threshold만 크게
test_hyperparameters(filepath, std_threshold=5.0, confidence_threshold=0.7, window_size=5)

# 조합 3: std_threshold 더 크게
test_hyperparameters(filepath, std_threshold=10.0, confidence_threshold=0.7, window_size=5)

# 조합 4: 모든 파라미터 관대하게
test_hyperparameters(filepath, std_threshold=20.0, confidence_threshold=0.3, window_size=15)

# 조합 5: 극단적으로 관대하게
test_hyperparameters(filepath, std_threshold=50.0, confidence_threshold=0.1, window_size=25)

In [119]:
test_hyperparameters(filepath, std_threshold=50.0, confidence_threshold=0.1, window_size=25)


=== 하이퍼파라미터 테스트 ===
std_threshold: 50.0
confidence_threshold: 0.1
window_size: 25
오디오 로드: 12.03초
SPICE 모델 로딩 중...
SPICE 모델 로드 완료
오디오 텐서 생성: shape=(192512,), dtype=<dtype: 'float32'>
SPICE 모델 변수 초기화 중...
변수 초기화 중...
초기화 성공
SPICE 모델 실행 중...
SPICE 모델 실행 완료:
  - 피치 배열 크기: 377
  - 평균 신뢰도: 0.440
  - 유효한 피치 개수: 377
필터링: 377개 중 321개 유효

=== 결과 ===
단일음정 프레임: 321개/377개
단일음정 시간: 10.24초
안정성 비율: 85.15%
목표(10초)와의 차이: 0.24초


2025-08-02 06:17:02.319541: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


(10.244753315649866, 0.8514588859416445)